<a href="https://colab.research.google.com/github/hevertonvalerio/Analise_Exploratoria/blob/main/analise_setorial_Hev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Relatório 01 - Análise Setorial de Ativos - Bloomberg Challenge
---

O objetivo desse relatório é identificar setores e ativos a serem analisados e investidos ao longo do Bloomberg Challenge de Outubro/2024.

## 1. Bibliotecas e Setup

### 1.1. Bibliotecas utilizadas

In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import matplotlib.pyplot as plt

import yfinance as yf
import requests
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import numpy as np
import os

### 1.2. Importação de dados das ativos passíveis de investir

Para esse passo, foi utilizado o arquivo "WLS as of Sep 25 20241.xlsx", compartilhado no grupo Mack IA Finance. Esse arquivo contém cerca de 10 mil ativos possivelmente a serem investidos.


#### 1.2.1. Enriquecimento dos dados de ativos passíveis de investir

Como não havia inicialmente o setor nem o nome do ticker de cada um dos ativos, foi necessária a criação de algumas funções para extração de dados do Yahoo Finance via url e posterior enriquecimento do arquivo.


In [4]:
def search_ticker(company_name):
    """ Função para extrair o symbol e outras informações para cada ticker """
    url = "https://query1.finance.yahoo.com/v1/finance/search"

    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:122.0) Gecko/20100101 Firefox/122.0"
    }

    params = {
        "q": f"{company_name}",
        "lang": "en-US",
        "region": "US",
    }

    try:
        data = requests.get(url, params=params, headers=headers).json()
        df = pd.DataFrame(data["quotes"])
        first_quote = df.iloc[0]
        return first_quote.get('symbol', 'N/A'), first_quote.get('sector', 'N/A'), first_quote.get('industry', 'N/A'), first_quote.get('shortname', 'N/A')
    except:
        print(f"Não foi encontrado info para {company_name}")
        return 'N/A', 'N/A', 'N/A', 'N/A'

def add_tickers_to_excel(input_file, output_file):
    """ Função para enriquecer o arquivo original """

    df = pd.read_excel(input_file)

    df['MainTicker'] = [x.split(' ')[0] for x in df['Ticker']]  # Utilizei o primeiro nome da coluna Ticker
    df[['YFTicker', 'SetorEconomico', 'Industria', 'NomeCompletoParaAuditoria']] = df['MainTicker'].apply(lambda x: pd.Series(search_ticker(x)))

    # Retorna o Excel enriquecido
    df.to_excel(output_file, index=False)
    print(f"Updated file saved as {output_file}")

# Determinar o input e o nome do arquivo enriquecido
input_file = 'WSL as of Sep 25 20241.xlsx'
output_file = 'WSL as of Sep 25 20241 - Modificado.xlsx'

# Enriquecer o arquivo
add_tickers_to_excel(input_file, output_file)


FileNotFoundError: [Errno 2] No such file or directory: 'WSL as of Sep 25 20241.xlsx'

Tratando os dados para excluir os ativos não encontrados no Yahoo Finance

In [ ]:
ativos = pd.read_excel(r'WSL as of Sep 25 20241 - Modificado.xlsx')
ativos = ativos[~ativos['YFTicker'].isna()]
ativos.head()

Além do Setor Econômico, seria interessante também identificar a bolsa/país principal em que o ativo é negociado, além da moeda utilizada.

In [ ]:
h = 0

def add_more_infos(company_name):
    """ Função para adicionar o país em que o ativo é operado e a moeda utilizada"""
    global h

    h += 1
    try:
        ticker = yf.Ticker(company_name)
    except:
        print(company_name)
        raise
    country = ticker.info.get('country', 'N/A')
    currency = ticker.info.get('currency', 'N/A')
    if h % 100 == 0:
        print(h)
    return country, currency

def add_infos_to_excel(ativos, output_file):
    """ Função para enriquecer o arquivo original """
    ativos[['Pais', 'Moeda']] = ativos['YFTicker'].apply(lambda x: pd.Series(add_more_infos(x)))

    # Retorna o Excel enriquecido
    ativos.to_excel(output_file, index=False)
    print(f"Updated file saved as {output_file}")

output_file = 'WSL as of Sep 25 20241 - Modificado_2.xlsx'

add_infos_to_excel(ativos, output_file)

#### 1.2.2. Importação de dados OHLCV

Após realizar um enriquecimento dos ativos, será realizada uma extração dos dados OHLCV <i>Open, High, Low, Close, Volume</i>, especificamente os dados de <i>Close</i> e <i>Volume</i>.

Para permitir uma análise setorial, ao invés de usar os dados especificamente de cada ativo, será realizado um tratamento em fluxo, por meio do qual se buscará extrair:
- Variação diária
- Volume

Diante da variação diária, se buscará reduzir as informações para apenas um vetor de variação diária por indústria / moeda / país.

In [ ]:
ativos = pd.read_excel(r'WSL as of Sep 25 20241 - Modificado_2.xlsx')
ativos = ativos[~ativos['Pais'].isna()] # Filtrando por ativos que possuam informações de Pais
ativos = ativos[~ativos['Moeda'].isna()] # Filtrando por ativos que possuam informações de moeda
ativos.head()

Para identificar esse vetor único, adotei a seguinte estatística/passo-a-passo:

1. Obter volume financeiro movimentado no dia pelo ativo (${Volume_{FinanceiroDiarioAtivo}}$)
2. Obter variação do valor de fechamento diário (${VariacaooFechamento_{Ativo}}$)
3. Multiplicar os dois primeiros valores (1 e 2)
4. Obter o volume financeiro total por dia no setor do ativo (${Volume_{FinanceiroDiarioTotal}}$)
5. Obter a soma das variações sopesadas pelo volume financeiro (item 3)
6. Dividir o item 5 pelo item 4

Com isso, espera-se encontrar uma estatística de variação diária do preço de fechamento do setor (sopesada pelo volume financeiro).

$$ {VariacaoDiaria_{Ponderada}} = \frac{\sum_{k=1}^n {Volume_{FinanceiroDiarioAtivo}} * {VariacaoFechamento_{Ativo}}}{{Volume_{FinanceiroDiarioTotal}}}  $$

In [ ]:
def obtain_sector_performance(lista_symbols):
    """ Função para realizar extract, transform dos dados em fluxo, transformando-os em vetores de rendimento por setor """
    data_inicial = dt(2002, 1, 1)
    data_final = dt(2024, 10, 1)

    dados_setoriais = pd.DataFrame()

    # Obtendo o vetor representativo do segmento por ano
    while data_inicial <= data_final:
        data_chunk = min(data_inicial + relativedelta(years=1), data_final)
        volume_total = pd.Series()
        dados_anuais = pd.DataFrame()

        for symbol in lista_symbols:
            data = yf.download(symbol, start=data_inicial.strftime('%Y-%m-%d'), end=data_chunk.strftime('%Y-%m-%d'), progress=False)[['Close', 'Volume']]

            if not data.empty:

                # Evitar SettingWithCopyWarning
                data = data.copy()

                # Calcular volume financeiro
                data['DollarVolume'] = data['Volume'] * data['Close']

                # Calcular retorno com ponderação por volume financeiro
                data[symbol] = data['Close'].pct_change() * data['DollarVolume']

                # Acumular volume financeiro
                volume_total = volume_total.add(data['DollarVolume'], fill_value=0)

                # Adicionar o valor do retorno ponderado ao pandas
                dados_anuais = pd.concat([dados_anuais, data[[symbol]]], axis=1)

        volume_total.replace(0, np.nan, inplace=True)

        # Obtendo indicador comparativo de cada setor por ano
        dados_anuais['RetornoSetor'] = dados_anuais.sum(axis=1) / volume_total # Usei a média mensal de volume financeiro movimentado por todo o setor

        # Juntando num só dataframe
        dados_setoriais = pd.concat([dados_setoriais, dados_anuais[['RetornoSetor']]])

        # Passando para o próximo ano
        data_inicial += relativedelta(years=1)

    return dados_setoriais

In [ ]:
def pipeline(ativos):
    """ Função para executar o pipeline do ETL """
    print("Iniciando pipeline")
    for moeda in ativos['Moeda'].unique():
        for pais in ativos[ativos['Moeda']==moeda]['Pais'].unique():
            for industria in ativos[(ativos['Moeda']==moeda)&(ativos['Pais']==pais)]['Industria'].unique():
                if f'{moeda}_{pais}_{industria}_setoriado.csv' not in os.listdir('data'):
                    ativos_chunk = ativos[(ativos['Moeda']==moeda)&(ativos['Pais']==pais)&(ativos['Industria']==industria)]

                    if not ativos_chunk.empty:
                        lista_ativos = list(ativos_chunk['YFTicker'])
                        temp = obtain_sector_performance(lista_ativos)
                        temp.to_csv(f'data\{moeda}_{pais}_{industria}_setoriado.csv')
                        print(f"Concluído {moeda}_{pais}_{industria}")
    print("Pipeline concluído!")
    return None

Executando o pipeline:

In [ ]:
pipeline(ativos)

## 2. Análise Exploratória de Dados

Considerando os dados extraídos e transformados nas seções anteriores, inicia-se a análise exploratória.

### 2.1. Ativos que compõem os setores

In [ ]:
ativos.info()

Percebe-se acima que há ativos para os quais as funções de enriquecimento não encontraram a correlata Industria. Esses ativos serão desconsiderados.
Além disso, deve-se tornara a coluna "Ponderação" de tipo "float".

Por meio de iterações, percebeu-se a existência de uma inconsistência na coluna Ponderação, conforme abaixo. Optou-se por tratá-la, modificando-a para 0

In [ ]:
display(ativos[ativos['Ponderação']=='--'])
ativos.loc[ativos['Ponderação']=='--', 'Ponderação'] = 0

ativos['Ponderação'] = ativos['Ponderação'].astype(float)

In [ ]:
ativos.dropna(inplace=True)

### 2.2. Análise de participação de mercado financeiro

In [ ]:
fig = go.Figure()


contagem_paises = ativos.Pais.value_counts()[:10].sort_values(ascending=True)
fig.add_trace(go.Bar(y = contagem_paises.index, x = contagem_paises.values, orientation='h'))

fig.update_xaxes(title_text='<b> Quantidade de ativos')

fig.update_layout(title_text='Distribuição de ativos por países - Top 10', width=800, height=600)
fig.show()

Há uma predominância de ativos listados nas bolsas dos Estados Unidos.

In [ ]:
fig = go.Figure()


contagem_paises = ativos.SetorEconomico.value_counts()[:30].sort_values(ascending=True)
fig.add_trace(go.Bar(y = contagem_paises.index, x = contagem_paises.values, orientation='h'))

fig.update_xaxes(title_text='<b> Quantidade de ativos')

fig.update_layout(title_text='Distribuição de ativos por Setores Econômicos', width=800, height=600)
fig.show()

In [ ]:
fig = go.Figure()


contagem_paises = ativos.Industria.value_counts()[:30].sort_values(ascending=True)
fig.add_trace(go.Bar(y = contagem_paises.index, x = contagem_paises.values, orientation='h'))

fig.update_xaxes(title_text='<b> Quantidade de ativos')

fig.update_layout(title_text='Distribuição de ativos por Indústria', width=800, height=600)
fig.show()

Essa análise, contudo, não leva em consideração o volume financeiro transacionado, o que é importante para o estudo. Deve-se modificar o código para obter a participação de cada indústira/setor/país.

In [ ]:
part_industria = ativos.groupby(['Industria'])['Ponderação'].sum()

In [ ]:
fig = go.Figure()

part_industria.sort_values(ascending=True, inplace=True)
fig.add_trace(go.Bar(y = part_industria.index[-10:], x = part_industria.values[-10:], orientation='h'))

fig.update_xaxes(title_text='<b> Participação no Mercado Global')

fig.update_layout(title_text='Distribuição de ativos por Indústria - Top 10', width=800, height=600)
fig.show()

In [ ]:
fig = go.Figure()

part_setor = ativos.groupby(['SetorEconomico'])['Ponderação'].sum()
part_setor.sort_values(ascending=True, inplace=True)
fig.add_trace(go.Bar(y = part_setor.index, x = part_setor.values, orientation='h'))

fig.update_xaxes(title_text='<b> Participação no Mercado Global')

fig.update_layout(title_text='Distribuição de ativos por Setor Econômico', width=800, height=600)
fig.show()

O resultado não pode ser interpretado como percentual ou valor absoluto: o campo <i>Ponderação</i>, proporcionado pelo arquivo Excel inicialmente compartilhado com o grupo MackIA, não aparenta se referir ao percentual de participação global, sendo possível que represente o percentual de suas bolsas de valores.

Ainda assim, a prevalência de determinados setores e indústrias nos permite concluir sobre a importância desses setores, em geral, por terem maiores participações "em cada bolsa de valores".

### 2.3. Análise de Relacionamento entre setores

A fim de descobrir possível dependência entre setores (e reduzir o risco pela seleção de setores menos correlacionados), prossegue-se a verificação dos vetores de cada país/moeda/indústria.

In [ ]:
lista_csv = os.listdir('data')
arquivos_csv_organizados = {'Moeda':[], 'Pais':[], 'Industria': [], 'Endereco': []}

for arquivo in lista_csv:
    dados = arquivo.split('_')
    arquivos_csv_organizados['Moeda'].append(dados[0])
    arquivos_csv_organizados['Pais'].append(dados[1])
    arquivos_csv_organizados['Industria'].append(dados[2])
    arquivos_csv_organizados['Endereco'].append(arquivo)

arquivos_organizados = pd.DataFrame(arquivos_csv_organizados)

In [ ]:
arquivos_organizados

#### 2.3.1. Análise de setores - EUA

In [ ]:
dados = pd.DataFrame()

arquivos_a_importar = arquivos_organizados[arquivos_organizados['Pais']=='United States']['Endereco']

for arquivo in arquivos_a_importar:
    temp = pd.read_csv(rf'data\{arquivo}', parse_dates=[0], index_col=[0])
    # temp.set_index(temp['Date'], inplace=True)
    temp = temp[['RetornoSetor']]
    temp.columns = [arquivo.split('_')[2]]
    temp.columns = [arquivo.split('_')[2]]
    dados = pd.concat([dados, temp], axis=1)
    # break

dados.head()

In [ ]:
dados.isna()